## Prueba de implementación de PoinNet

Este _notebook_ está dedicado a probar una **implementación de PointNet con Tensorflow 2.0 y Keras**. Se utilizará un subconjunto de validación del **GOOSE dataset** para entrenar el modelo. Este consta de **151 nubes de puntos** de una dimensión variable. Se realizará un estudio estadístico de las clases contenidas en estas 151 nubes de puntos para evaluar si es necesario completar con más nubes de puntos que contengan las clases faltantes.

### Importaciones

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import sys
import re
import matplotlib.pyplot as plt
import random as rand


from pathlib import Path
from sklearn.model_selection import train_test_split

### Carga del GOOSE Dataset

In [3]:
# Windows

# point_clouds_path = Path(r"goose\val\2022-07-22_flight")
# labels_path = Path(r"goose\labels\val\2022-07-22_flight")

# Linux

point_clouds_path = Path(r"/home/felix/Escritorio/TFG/datasets/Goose/goose_3d_val/lidar/val/2022-07-22_flight")
labels_path = Path(r"/home/felix/Escritorio/TFG/datasets/Goose/goose_3d_val/labels/val/2022-07-22_flight")

Se ordenan los archivos para establecer un orden entre nubes de puntos y etiquetas

In [4]:
files_list = os.listdir(point_clouds_path)
labels_list = os.listdir(labels_path)

sorted_files = sorted(files_list, key=lambda x: int(re.search(r'__\d{4,5}_', x).group(0)[2:-1]))
sorted_labels = sorted(labels_list, key=lambda x: int(re.search(r'__\d{4,5}_', x).group(0)[2:-1]))

### Extracción de caracteristicas del Dataset Bruto

Extracción del minimo y el máximo de numero de puntos en las 151 muestras LiDAR

In [ ]:
list_len_point_clouds = []

# Reading .bin files and adding to DF
for file in os.listdir(point_clouds_path):
    scan = np.fromfile(os.path.join(point_clouds_path, file), dtype=np.float32)
    scan = scan.reshape((-1, 4))

    # put in attribute
    points = scan[:, 0:3]    # get xyz
    remissions = scan[:, 3]  # get remission

    list_len_point_clouds.append(len(points))

list_len_point_clouds = np.array(list_len_point_clouds)

min_idx = list_len_point_clouds.argmin() 
max_idx = list_len_point_clouds.argmax()

min = list_len_point_clouds[min_idx]
max = list_len_point_clouds[max_idx]

print(f"La nube con más puntos tiene: {max}")
print(f"La nube con menos puntos tiene: {min}")

### Extracción de las clases existentes y su numero de puntos

In [ ]:
list_labels = []


for file in os.listdir(labels_path):

    # reading a .label file
    label = np.fromfile(os.path.join(labels_path, file), dtype=np.uint32)
    label = label.reshape((-1))

    # extract the semantic and instance label IDs
    sem_label = label & 0xFFFF  # semantic label in lower half

    list_labels.append(pd.DataFrame(sem_label, columns=["sem_label"]))

df_list_labels = pd.concat(list_labels)
    
# df_list_labels

plt.figure(figsize=(16, 6))
plt.hist(df_list_labels['sem_label'], 
         bins=np.arange(df_list_labels['sem_label'].min(), df_list_labels['sem_label'].max() + 2), 
         edgecolor='k', 
         alpha=0.7)

# Personalización del gráfico
plt.title("Histograma de Etiquetas Semánticas")
plt.xlabel("Etiqueta Semántica")
plt.ylabel("Frecuencia")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Configurar las marcas del eje X en incrementos de 5
x_ticks = np.arange(df_list_labels['sem_label'].min(), df_list_labels['sem_label'].max() + 1, 5)
plt.xticks(x_ticks)

plt.show()

Ahora guardamos en un diccionario la **frecuencia** de cada clase semántica 

In [ ]:
dict_freq_classes = df_list_labels['sem_label'].value_counts().sort_index().to_dict()

print(dict_freq_classes)

### Creación del Dataset de entrenamiento

Se leen los puntos de cada muestra y las etiquetas de cada muestra. `list_point_clouds` **es una lista** que almacena las 151 nubes de puntos, cada una es un dataframe con las columnas `x`, `y`, `z`, `remissions`

In [8]:
list_point_clouds = []

# Reading .bin files and adding to DF
for file in os.listdir(point_clouds_path):
    scan = np.fromfile(os.path.join(point_clouds_path, file), dtype=np.float32)
    scan = scan.reshape((-1, 4))

    # put in attribute
    points = scan[:, 0:3]    # get xyz
    remissions = scan[:, 3]  # get remission

    df_point_cloud = pd.DataFrame(points, columns=["x","y","z"])
    df_point_cloud["remissions"] = remissions
    list_point_clouds.append(df_point_cloud)

# print(list_point_clouds[0:2])
# print(list_labels[0:2])

del df_point_cloud

En la siguiente celda se imprime por pantalla ***el DataFrame*** que representa la nube de puntos **nº40**

In [ ]:
list_point_clouds[39]

### Normalización

Para evitar que la normalización se contamine del ruido que pueden contener algunos valores de la nube de puntos se realiza una **normalización de media 0 y varianza 1** para las `remisiones` y una normalización que redimensione la nube de puntos a una esfera unitaria para las entradas geométricas `x`, `y` y `z` 

En primer lugar dividimos nuestro conjunto en subconjuntos de entrenamiento y validación
- Se normalizan ambos subconjuntos en función a esos estadísticos

In [35]:
# División: train, val

x_train, x_val, y_train, y_val = train_test_split(list_point_clouds, list_labels, test_size=0.2, random_state=42) # random_state para estabilidad en los experimentos

print(f'Hay {len(x_train)} nubes de puntos en el conjunto de entrenamiento')
print(f'Hay {len(x_val)} nubes de puntos en el conjunto de validación')


Hay 120 nubes de puntos en el conjunto de entrenamiento
Hay 31 nubes de puntos en el conjunto de validación


Se calculan los estadísticos de las remisiones del conjunto de entrenamiento y l

In [37]:
# Cálculo de los estadísticos de x_train

x_train_concat = pd.concat(x_train)

mean = x_train_concat["remissions"].mean()
std = x_train_concat["remissions"].std()

print(f'Las media de las remisiones son: \n\n{means}\n')
print(f'Las desviación estandar de las remisiones son: \n\n{stds}')

Las media de las remisiones son: 

29.32798194885254

Las desviación estandar de las remisiones son: 

20.90127944946289


In [32]:
# Eliminar outliers usando percentiles

x_train_concat['distance'] = np.sqrt(x_train_concat['x']**2 + x_train_concat['y']**2 + x_train_concat['z']**2)

# 2. Calcular el percentil deseado (por ejemplo, 99%)
threshold = np.percentile(x_train_concat['distance'], 97)

print(threshold)

x_train_concat_filtered = x_train_concat[x_train_concat['distance'] <= threshold].copy()

d_max = np.sqrt(x_train_concat_filtered['x']**2 + x_train_concat_filtered['y']**2 + x_train_concat_filtered['z']**2).max()

print(d_max)

113.732
113.732


Normalización de caracteristicas

In [42]:
norm_list_point_clouds = []

for df in list_point_clouds:
    norm_df = df.copy()
    norm_df[['x','y','z']] = (df[['x','y','z']] / d_max)
    norm_df['remissions'] = (norm_df['remissions'] - mean) / std
    norm_list_point_clouds.append(norm_df)

norm_list_point_clouds[0]


,x,y,z,remissions
0,-0.010641,-0.001212,-0.004994,-0.398444
1,-0.054726,-0.006168,-0.019591,-1.259635
2,-0.068293,-0.007624,-0.019759,-0.398444
3,-0.082959,-0.008778,-0.020128,-0.494132
4,-0.096179,-0.011503,-0.020134,-0.541975
...,...,...,...,...
181216,-0.436390,-0.051882,0.044096,0.127840
181217,-0.429196,-0.050495,0.046032,-0.685507
181218,-0.448852,-0.052252,0.051326,-0.015692
181219,-0.438318,-0.050560,0.054019,-0.541975


A continuación se guardan en archivos csv

In [43]:
output_dir = "/home/felix/Escritorio/TFG/datasets_norm/goose_norm_test"
os.makedirs(output_dir, exist_ok=True)

# Guardar cada DataFrame en un archivo
for i, df in enumerate(norm_list_point_clouds):
    # Define el nombre del archivo, por ejemplo: dataframe_0.csv
    file_name = f"dataframe_{i}.csv"  # Cambia a .parquet si prefieres parquet
    file_path = os.path.join(output_dir, file_name)
    
    # Guardar el DataFrame como CSV
    df.to_csv(file_path, index=False)  # Usa index=False para omitir el índice
    print(f"Guardado: {file_path}")


Guardado: ./data/goose_norm_files/dataframe_0.csv
Guardado: ./data/goose_norm_files/dataframe_1.csv
Guardado: ./data/goose_norm_files/dataframe_2.csv
Guardado: ./data/goose_norm_files/dataframe_3.csv
Guardado: ./data/goose_norm_files/dataframe_4.csv
Guardado: ./data/goose_norm_files/dataframe_5.csv
Guardado: ./data/goose_norm_files/dataframe_6.csv
Guardado: ./data/goose_norm_files/dataframe_7.csv
Guardado: ./data/goose_norm_files/dataframe_8.csv
Guardado: ./data/goose_norm_files/dataframe_9.csv
Guardado: ./data/goose_norm_files/dataframe_10.csv
Guardado: ./data/goose_norm_files/dataframe_11.csv
Guardado: ./data/goose_norm_files/dataframe_12.csv
Guardado: ./data/goose_norm_files/dataframe_13.csv
Guardado: ./data/goose_norm_files/dataframe_14.csv
Guardado: ./data/goose_norm_files/dataframe_15.csv
Guardado: ./data/goose_norm_files/dataframe_16.csv
Guardado: ./data/goose_norm_files/dataframe_17.csv
Guardado: ./data/goose_norm_files/dataframe_18.csv
Guardado: ./data/goose_norm_files/datafra